## 1. DataFrames creation, light EDA & some data cleaning

In [361]:
import pandas as pd

# df_http
df_http = pd.read_csv('./data/input/polling.csv', index_col = 0)
print(df_http.shape)
# print(df_http.info())
df_http.head()

(350445, 4)


,creation_time,device_id,error_code,status_code
0,2020-02-26 19:16:40.481,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
1,2020-02-26 19:16:25.035,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
2,2020-02-26 18:31:52.126,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
3,2020-02-26 18:31:17.043,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
4,2020-02-26 18:31:01.738,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200


In [362]:
# df_status
df_status = pd.read_csv('./data/input/connectivity_status.csv', index_col = 0)
print(df_status.shape)
df_status.head()

(35559, 3)


,creation_time,status,device_id
0,2020-02-26 19:31:29.998,OFFLINE,00083c70-7f54-4324-94bd-b0cb0be78baf
1,2020-02-26 19:31:39.677,ONLINE,00083c70-7f54-4324-94bd-b0cb0be78baf
2,2020-02-26 20:08:42.789,OFFLINE,000fec74-a5b4-40fc-b93c-eef3b2afb87e
3,2020-02-26 20:09:03.448,ONLINE,000fec74-a5b4-40fc-b93c-eef3b2afb87e
4,2020-02-26 00:30:03.439,OFFLINE,001d3c67-99fd-43b7-8207-f420fa303a49


In [363]:
# df_orders
df_orders = pd.read_csv('./data/input/orders.csv', index_col = 0)

print(f"Orders count pre-cleaning: {df_orders.shape[0]}")
# we should get rid of rows with no device_id associated since we're focusing on metrics related to dispatched orders only
print(f"Orders count not dispatched to a device: {len(df_orders.loc[df_orders['device_id'].isna(), 'order_id'].unique())}")

df_orders = df_orders.loc[df_orders.device_id.notna()]
print(f"Orders count post-cleaning: {df_orders.shape[0]}")
df_orders.head()

Orders count pre-cleaning: 2357
Orders count not dispatched to a device: 26
Orders count post-cleaning: 2331


,order_creation_time,order_id,device_id
5,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862
6,2020-02-26 10:34:36,102492697,2bb11f99-ab21-4628-abe6-b919da8fbf34
7,2020-02-26 11:32:47,102500373,2aec0e20-e1d8-4323-9b12-f066856488a7
8,2020-02-26 11:46:17,102503173,37638585-a181-4265-aeb2-d9e284bb30c3
9,2020-02-26 11:38:40,102501909,53d16c33-5980-4ad8-9032-f85d686d2855


----

## 2. Data manipulation

In [364]:
import random

df = pd.merge(df_orders, df_http, on = 'device_id', how = 'left', indicator = True)
display(df.head())

print(f"Orders count dispatched to unknown devices: {df.groupby('_merge').size()['left_only']}") # associated device_id is missing from the polling dataset
inc_orders = list(df.loc[df._merge == 'left_only', 'order_id'].unique()) # here's the dataset (as a list) of inconsistent orders for further analysis
print(f"Here's one of those oders to eventually double check its metrics behavior later on down the pipeline: {inc_orders[random.randint(0, len(inc_orders))]}")

,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:28:41.657,NaN,200.0,both
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:51.879,NaN,200.0,both
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:35.925,NaN,200.0,both
4,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:26:34.578,NaN,200.0,both


Orders count dispatched to unknown devices: 71
Here's one of those oders to eventually double check its metrics behavior later on down the pipeline: 102621973


In [365]:
df[['order_creation_time', 'creation_time']] = df[['order_creation_time', 'creation_time']].apply(pd.to_datetime) # convert time columns to DateTime type
print(df.shape)
df.head()

(4710527, 7)


,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:28:41.657,NaN,200.0,both
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:51.879,NaN,200.0,both
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:35.925,NaN,200.0,both
4,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:26:34.578,NaN,200.0,both


In [366]:
def cut_df(
    df: pd.DataFrame,
    m_os: int
) -> pd.DataFrame:
    """Slice a DataFrame over a specific portion of time.

    Args:
        df (pd.DataFrame): a DataFrame to be sliced.
        m_os (int): a time offset (minutes).
    
    Returns:
        df_cut (pd.DataFrame): the slice of the DataFrame whose rows
            are comprised between the order creation time and:
                - the preceding time segment determined by the offset (if m_os < 0)
                - the following time segment determined by the offset (if m_os > 0)
    
    Raises:
        ValueError: `m_os` param has to be either positive
            or negative, not a zero time segment (0).
    """
    if m_os < 0:
        df_cut = df.loc[df['creation_time'].between(df['order_creation_time'] + pd.DateOffset(minutes = m_os), df['order_creation_time'])]
        df_cut['timeslice'] = f"{abs(m_os)}m_before"

    elif m_os > 0:
        df_cut = df.loc[df['creation_time'].between(df['order_creation_time'], df['order_creation_time'] + pd.DateOffset(minutes = m_os))]
        df_cut['timeslice'] = f"{abs(m_os)}m_after"

    else:
        raise ValueError('`m_os` param cannot be 0')
    return df_cut

In [367]:
pd.options.mode.chained_assignment = None # this prevents the good old SettingWithCopyWarning to jump out in the cell output

slices = [-3, -60, 3]

df_list = [cut_df(df, i) for i in slices]
df_raw = pd.concat(df_list, axis = 0, ignore_index = True) # slices union (row-wise concat)

print(df_raw.shape)
df_raw.loc[df_raw.order_id == 102452116].head()

(354527, 8)


,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge,timeslice
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:08:12.125,ECONNABORTED,0.0,both,3m_before
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:06:46.961,NaN,200.0,both,3m_before
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:06:20.304,NaN,200.0,both,3m_before
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:06:18.986,NaN,200.0,both,3m_before
16479,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:08:12.125,ECONNABORTED,0.0,both,60m_before


In [368]:
# dropna = False is required since groupby() is by default filtering out those groups whose aggregating columns are NaN
df_report = df_raw.groupby(['order_id', 'timeslice', 'status_code', 'error_code'], dropna = False).size().reset_index(name = 'rows_count')
df_report.head()
# resulting df now contains counts for all the possible combinations of our 4 grouping features: this will be our report baseline (now to be enriched with some other KPIs)

,order_id,timeslice,status_code,error_code,rows_count
0,102452116,3m_after,200.0,NaN,1
1,102452116,3m_before,0.0,ECONNABORTED,1
2,102452116,3m_before,200.0,NaN,3
3,102452116,60m_before,0.0,ECONNABORTED,1
4,102452116,60m_before,200.0,NaN,31


In [369]:
# here we can see that those 71 orders dispatched to unknown devices have been implicitly removed with the time cutting operations
df_raw.loc[df.order_id.isin(inc_orders)]

,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge,timeslice


In [370]:
len(pd.unique(df_report.order_id)) # 1978 / 2331

1978

### 2.1 Computing closest polling timestamps

In [371]:
# immediately preceding and following order creation time events have to be computed across an unbounded period of time, that's why I'm using the first merged dataset: 'df'

# sorting here is somehow mandatory since later I'll use head(1) to pick up the two closest rows (before and after) to my time pivot point ('order_creation_time')
df_clst = df[['order_id', 'order_creation_time', 'creation_time']].sort_values(['order_id', 'creation_time'], ascending = [True, False])
df_clst['abs_dt_diff'] = abs(df['order_creation_time'] - df['creation_time'])
df_clst['poll_before_order'] = (df['order_creation_time'] > df['creation_time'])

df_clst.head()

,order_id,order_creation_time,creation_time,abs_dt_diff,poll_before_order
0,102452116,2020-02-26 00:08:19,2020-02-26 05:29:34.363,0 days 05:21:15.363000,False
1,102452116,2020-02-26 00:08:19,2020-02-26 05:28:41.657,0 days 05:20:22.657000,False
2,102452116,2020-02-26 00:08:19,2020-02-26 05:27:51.879,0 days 05:19:32.879000,False
3,102452116,2020-02-26 00:08:19,2020-02-26 05:27:35.925,0 days 05:19:16.925000,False
4,102452116,2020-02-26 00:08:19,2020-02-26 05:26:34.578,0 days 05:18:15.578000,False


In [372]:
df_clst_sg = df_clst.groupby(['order_id', 'poll_before_order']).head(1)
df_clst_sg.head()

,order_id,order_creation_time,creation_time,abs_dt_diff,poll_before_order
0,102452116,2020-02-26 00:08:19,2020-02-26 05:29:34.363,0 days 05:21:15.363000,False
751,102452116,2020-02-26 00:08:19,2020-02-26 00:08:12.125,0 days 00:00:06.875000,True
3332698,102452190,2020-02-26 00:08:41,2020-02-26 05:29:34.363,0 days 05:20:53.363000,False
3333449,102452190,2020-02-26 00:08:41,2020-02-26 00:08:12.125,0 days 00:00:28.875000,True
4101120,102453036,2020-02-26 00:13:57,2020-02-26 05:29:34.363,0 days 05:15:37.363000,False


In [373]:
df_clst_fl = df_clst_sg.pivot_table(index = ['order_id'], columns = 'poll_before_order', values = 'creation_time', aggfunc = max)
df_clst_fl.columns = ['after_poll_ts', 'before_poll_ts']

df_clst_fl.head()

,after_poll_ts,before_poll_ts
order_id,,
102452116,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
102452190,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
102453036,2020-02-26 05:29:34.363,2020-02-26 00:13:12.644
102453649,2020-02-26 05:29:34.363,2020-02-26 00:17:29.623
102453774,2020-02-26 05:29:34.363,2020-02-26 00:18:06.815


In [374]:
# and again, those same 71 orders are not in the resulting dataset since they don't have any polling timestamps
df_clst_fl.loc[df_clst_fl.index.isin(inc_orders)]

,after_poll_ts,before_poll_ts
order_id,,


In [375]:
len(pd.unique(df_clst_fl.index)) # 2260 / 2331 orders (71)

2260

### 2.2 Computing most recent connectivity status before an order (and its timestamp)

In [376]:
# orders not dispatched to a device_id have been already removed from df_orders
df_mrs_raw = pd.merge(
    pd.merge(df_orders, df_http, on = 'device_id', how = 'left', indicator = 'order_to_http'),
    df_status, on = 'device_id', how = 'left', indicator = 'http_to_status'
    )

df_mrs_raw.head()

,order_creation_time,order_id,device_id,creation_time_x,error_code,status_code,order_to_http,creation_time_y,status,http_to_status
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:18.764,OFFLINE,both
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:18.767,OFFLINE,both
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:19.568,ONLINE,both
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:19.575,ONLINE,both
4,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 02:08:52.638,OFFLINE,both


In [377]:
# as suggested in the assignment, not all devices have a connectivity status
print(df_mrs_raw.groupby('http_to_status').size())

print(f"Distinct order_id count across all the dataset: {len(pd.unique(df_mrs_raw.order_id))}")

nojoin_filter = ((df_mrs_raw['order_to_http'] == 'left_only') | (df_mrs_raw['http_to_status'] == 'left_only'))
print(f"Distinct order_id count consistent (intersecting) all across the 3 datasets: {len(df_mrs_raw.loc[~nojoin_filter, 'order_id'].unique())}")

df_mrs_raw.head() 

http_to_status
left_only     3504552
right_only          0
both          9629231
dtype: int64
Distinct order_id count across all the dataset: 2331
Distinct order_id count consistent (intersecting) all across the 3 datasets: 557


,order_creation_time,order_id,device_id,creation_time_x,error_code,status_code,order_to_http,creation_time_y,status,http_to_status
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:18.764,OFFLINE,both
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:18.767,OFFLINE,both
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:19.568,ONLINE,both
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 00:44:19.575,ONLINE,both
4,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both,2020-02-26 02:08:52.638,OFFLINE,both


In [378]:
def get_mr_status_info(
    *cols: pd.Series
) -> pd.DataFrame:
    """Produce an aggregated DataFrame, showing
        for each order id, a tuple made of:
        - the most recent connectivity status before the specific order
        - the status corresponding to that data point

    Args:
        *cols (pd.Series): input DataFrame columns, in this order:
            - 1st (0): id of the order
            - 2nd (1): order creation timestamp
            - 3rd (2): connectivity status acquisition timestamp
            - 4th (3): connectivity status acquired
    
    Returns:
        df (pd.DataFrame): the aggregated DataFrame.

    """
    df_tmp = pd.concat([*cols], axis = 1)
    df_tmp['is_status_ts_before_order_ts'] = (df_tmp[cols[1].name] > df_tmp[cols[2].name])
    df = (df_tmp.loc[df_tmp.is_status_ts_before_order_ts == True] # on those status update rows preceding order creation time only
        .groupby(cols[0].name)[[cols[2].name, cols[3].name]] # grouping by order_id, indexing on the columns I need
        .max(cols[3].name) # get the most recent status update timestamp
    )
    return df

In [379]:
df_mrs_fnl = get_mr_status_info(
    df_mrs_raw['order_id'],
    df_mrs_raw['order_creation_time'],
    df_mrs_raw['creation_time_y'],
    df_mrs_raw['status']
)

df_mrs_fnl.columns = ['conn_sta_acq_ts', 'conn_sta_acqd']
df_mrs_fnl.head()

,conn_sta_acq_ts,conn_sta_acqd
order_id,,
102458251,2020-02-26 00:44:19.575,ONLINE
102458819,2020-02-26 00:44:19.575,ONLINE
102463944,2020-02-26 00:44:19.575,ONLINE
102472308,2020-02-26 02:08:53.501,ONLINE
102472764,2020-02-26 02:08:53.501,ONLINE


In [380]:
len(pd.unique(df_mrs_fnl.index)) # 374 / 2331 orders

374

### 2.3 Putting the pieces together

In [381]:
import numpy as np

# computing status_code KPIs
df_pv_sc = df_report.pivot_table(index = ['order_id', 'timeslice'], columns = 'status_code', values = 'rows_count', aggfunc = np.sum)#, fill_value = 0)
df_pv_sc['polling_events'] = df_pv_sc.loc[:, list(df_pv_sc.columns)].sum(axis = 1) # creating the polling events count column as the sum of all the other columns
df_pv_sc.head()

status_code           0.0  200.0  401.0  polling_events
order_id  timeslice                                    
102452116 3m_after    NaN    1.0    NaN             1.0
          3m_before   1.0    3.0    NaN             4.0
          60m_before  1.0   31.0    NaN            32.0
102452190 3m_after    NaN    2.0    NaN             2.0
          3m_before   1.0    3.0    NaN             4.0

In [382]:
# computing error_code KPIs
df_pv_ec = df_report.pivot_table(index = ['order_id', 'timeslice'], columns = 'error_code', values = 'rows_count', aggfunc = np.sum)#, fill_value = 0)
# we don't need to compute count of responses without error codes. since it has to be equal to 200 responses count
df_pv_ec.head()

error_code            ECONNABORTED  GENERIC_ERROR
order_id  timeslice                              
102452116 3m_before            1.0            NaN
          60m_before           1.0            NaN
102452190 3m_before            1.0            NaN
          60m_before           1.0            NaN
102453036 60m_before           1.0            NaN

In [383]:
df_final = pd.concat([df_pv_sc, df_pv_ec], axis = 1)
df_final = df_final.fillna(0).astype(int)
df_final.head()

0.0  200.0  401.0  polling_events  ECONNABORTED  \
order_id  timeslice                                                     
102452116 3m_after      0      1      0               1             0   
          3m_before     1      3      0               4             1   
          60m_before    1     31      0              32             1   
102452190 3m_after      0      2      0               2             0   
          3m_before     1      3      0               4             1   

                      GENERIC_ERROR  
order_id  timeslice                  
102452116 3m_after                0  
          3m_before               0  
          60m_before              0  
102452190 3m_after                0  
          3m_before               0

In [384]:
df_report_csv_sg = df_final.join(df_clst_fl, how = 'left')
df_report_csv_sg.head()

0.0  200.0  401.0  polling_events  ECONNABORTED  \
order_id  timeslice                                                     
102452116 3m_after      0      1      0               1             0   
          3m_before     1      3      0               4             1   
          60m_before    1     31      0              32             1   
102452190 3m_after      0      2      0               2             0   
          3m_before     1      3      0               4             1   

                      GENERIC_ERROR           after_poll_ts  \
order_id  timeslice                                           
102452116 3m_after                0 2020-02-26 05:29:34.363   
          3m_before               0 2020-02-26 05:29:34.363   
          60m_before              0 2020-02-26 05:29:34.363   
102452190 3m_after                0 2020-02-26 05:29:34.363   
          3m_before               0 2020-02-26 05:29:34.363   

                              before_poll_ts  
order_id  timeslice                           
102452116 3m_after   2020-02-26 00:08:12.125  
          3m_before  2020-02-26 00:08:12.125  
          60m_before 2020-02-26 00:08:12.125  
102452190 3m_after   2020-02-26 00:08:12.125  
          3m_before  2020-02-26 00:08:12.125

In [385]:
df_report_csv_fl = df_report_csv_sg.join(df_mrs_fnl, how = 'left')
df_report_csv_fl.head()

0.0  200.0  401.0  polling_events  ECONNABORTED  \
order_id  timeslice                                                     
102452116 3m_after      0      1      0               1             0   
          3m_before     1      3      0               4             1   
          60m_before    1     31      0              32             1   
102452190 3m_after      0      2      0               2             0   
          3m_before     1      3      0               4             1   

                      GENERIC_ERROR           after_poll_ts  \
order_id  timeslice                                           
102452116 3m_after                0 2020-02-26 05:29:34.363   
          3m_before               0 2020-02-26 05:29:34.363   
          60m_before              0 2020-02-26 05:29:34.363   
102452190 3m_after                0 2020-02-26 05:29:34.363   
          3m_before               0 2020-02-26 05:29:34.363   

                              before_poll_ts conn_sta_acq_ts conn_sta_acqd  
order_id  timeslice                                                         
102452116 3m_after   2020-02-26 00:08:12.125             NaN           NaN  
          3m_before  2020-02-26 00:08:12.125             NaN           NaN  
          60m_before 2020-02-26 00:08:12.125             NaN           NaN  
102452190 3m_after   2020-02-26 00:08:12.125             NaN           NaN  
          3m_before  2020-02-26 00:08:12.125             NaN           NaN

In [386]:
len(pd.unique(df_report_csv_fl.index.get_level_values(0))) # 1978 / 2331

1978

----

##  3. Write CSV report

In [387]:
df_report_csv_fl.to_csv('./data/output/report.csv')